<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/Book_2/ch07/Chapter_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7 Language generate using RNN

**7.1 Language generate using Language Model**

*7.1.2 Langauge generate Implementation*

In [ ]:
import sys
sys.path.append('..')
